### Load preprocessed data

In [38]:
import pycuda.driver as cuda
import pycuda.autoinit # Necessary for using its functions
import numpy as np
cuda.init()
fh = np.load('../data/dataset.npz')

# We have a bunch of feature columns and last column is the y-target
# Note pytorch is finicky about need int64 types
train_x = fh['train_x'].astype(np.int64)
train_y = fh['train_y']

# We've already split into train & test
test_x = fh['test_x'].astype(np.int64)
test_y = fh['test_y']


n_user = int(fh['n_user'])
n_item = int(fh['n_item'])

# columns are user_id, item_id and other features 
# we won't use the 3rd and 4th columns
print(train_x)
print(' ')
print(train_y)

[[   1 1193   11   10]
 [   1  914   26   10]
 [   1 3408    7   10]
 ...
 [6040  562   37    6]
 [6040 1096  109    6]
 [6040 1097   99    6]]
 
[[5.]
 [3.]
 [4.]
 ...
 [5.]
 [4.]
 [4.]]


In [39]:
train_y.shape

(900227, 1)

### Define the MF Model

In [40]:
import torch
from torch import nn
import torch.nn.functional as F

In [41]:
class MF(nn.Module):
    itr = 0
    def __init__(self, n_user, n_item, k=18, c_vector=1.0, writer=None):
        super(MF, self).__init__()
        # This will hold the logging
        self.writer = writer
        
        # These are simple hyperparameters
        self.k = k
        self.n_user = n_user
        self.n_item = n_item
        self.c_vector = c_vector
        
        # These are learned and fit by PyTorch
        self.user = nn.Embedding(n_user, k)
        self.item = nn.Embedding(n_item, k)
    
    def __call__(self, train_x):
        # This is the most import function in this script
        # These are the user indices, and correspond to "u" variable
        user_id = train_x[:, 0]
        # Item indices, correspond to the "i" variable
        item_id = train_x[:, 1]
        # vector user = p_u
        vector_user = self.user(user_id)
        # vector item = q_i
        vector_item = self.item(item_id)
        # this is a dot product & a user-item interaction: p_u * q_i
        ui_interaction = torch.sum(vector_user * vector_item, dim=1)
        return ui_interaction
    
    def loss(self, prediction, target):
        # MSE error between target = R_ui and prediction = p_u * q_i
        loss_mse = F.mse_loss(prediction, target.squeeze())
        # Compute L2 reularization over user (P) and item (Q) matrices
        prior_user =  MF.l2_regularize(self.user.weight) * self.c_vector
        prior_item = MF.l2_regularize(self.item.weight) * self.c_vector
        # Add up the MSE loss + user & item regularization
        total = loss_mse + prior_user + prior_item
        
        # This logs all local variables to tensorboard
        for name, var in locals().items():
            if type(var) is torch.Tensor and var.nelement() == 1 and self.writer is not None:
                self.writer.add_scalar(name, var, self.itr)
        return total
    
    def l2_regularize(array):
        loss = torch.sum(array ** 2.0)
        return loss

### Train model

In [42]:
from ignite.engine import Events, create_supervised_trainer, create_supervised_evaluator
from ignite.metrics import Loss
from ignite.metrics import MeanSquaredError

from tensorboardX import SummaryWriter

from loader import Loader
from datetime import datetime

In [43]:
# Hyperparameters
lr = 1e-2
# Number of dimensions per user, item
k = 10
# regularization constant
c_vector = 1e-6

# Setup logging
log_dir = 'runs/simple_mf_01_' + str(datetime.now()).replace(' ', '_')
print(log_dir)
writer = SummaryWriter(logdir=log_dir)

runs/simple_mf_01_2019-06-08_21:27:41.679363


In [50]:
model = MF(n_user, n_item, writer=writer, k=k, c_vector=c_vector)
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

trainer = create_supervised_trainer(model, optimizer, model.loss)
metrics = {'accuracy': MeanSquaredError()}
evaluat = create_supervised_evaluator(model, metrics=metrics)

train_loader = Loader(train_x, train_y, batchsize=1024)
test_loader = Loader(test_x, test_y, batchsize=1024)

In [51]:
def log_training_loss(engine, log_interval=400):
    model.itr = engine.state.iteration
    if model.itr % log_interval == 0:
        fmt = "Epoch[{}] Iteration[{}/{}] Loss: {:.2f}"
        msg = fmt.format(engine.state.epoch, engine.state.iteration, 
                         len(train_loader), engine.state.output)
        print(msg)

trainer.add_event_handler(event_name=Events.ITERATION_COMPLETED, handler=log_training_loss)

def log_validation_results(engine):
    # When triggered, run the validation set
    evaluat.run(test_loader)
    avg_accuracy = evaluat.state.metrics['accuracy']
    print("Epoch[{}] Validation MSE: {:.2f} ".format(engine.state.epoch, avg_accuracy))
    writer.add_scalar("validation/avg_accuracy", avg_accuracy, engine.state.epoch)

trainer.add_event_handler(event_name=Events.EPOCH_COMPLETED, handler=log_validation_results)


#### Run model

In [52]:
trainer.run(train_loader, max_epochs=250)

Epoch[1] Iteration[400/879] Loss: 16.16
Epoch[1] Iteration[800/879] Loss: 3.37
Epoch[1] Validation MSE: 2.41 
Epoch[2] Iteration[1200/879] Loss: 1.30
Epoch[2] Iteration[1600/879] Loss: 1.15
Epoch[2] Validation MSE: 1.02 
Epoch[3] Iteration[2000/879] Loss: 0.95
Epoch[3] Iteration[2400/879] Loss: 0.94
Epoch[3] Validation MSE: 0.93 
Epoch[4] Iteration[2800/879] Loss: 0.94
Epoch[4] Iteration[3200/879] Loss: 0.90
Epoch[4] Validation MSE: 0.90 
Epoch[5] Iteration[3600/879] Loss: 0.81
Epoch[5] Iteration[4000/879] Loss: 0.80
Epoch[5] Validation MSE: 0.87 
Epoch[6] Iteration[4400/879] Loss: 0.81
Epoch[6] Iteration[4800/879] Loss: 0.86
Epoch[6] Iteration[5200/879] Loss: 0.84
Epoch[6] Validation MSE: 0.85 
Epoch[7] Iteration[5600/879] Loss: 0.82
Epoch[7] Iteration[6000/879] Loss: 0.84
Epoch[7] Validation MSE: 0.84 
Epoch[8] Iteration[6400/879] Loss: 0.83
Epoch[8] Iteration[6800/879] Loss: 0.81
Epoch[8] Validation MSE: 0.83 
Epoch[9] Iteration[7200/879] Loss: 0.78
Epoch[9] Iteration[7600/879] Loss

Epoch[67] Validation MSE: 0.78 
Epoch[68] Iteration[59200/879] Loss: 0.69
Epoch[68] Iteration[59600/879] Loss: 0.70
Epoch[68] Validation MSE: 0.78 
Epoch[69] Iteration[60000/879] Loss: 0.79
Epoch[69] Iteration[60400/879] Loss: 0.69
Epoch[69] Validation MSE: 0.78 
Epoch[70] Iteration[60800/879] Loss: 0.69
Epoch[70] Iteration[61200/879] Loss: 0.76
Epoch[70] Validation MSE: 0.78 
Epoch[71] Iteration[61600/879] Loss: 0.78
Epoch[71] Iteration[62000/879] Loss: 0.78
Epoch[71] Iteration[62400/879] Loss: 0.75
Epoch[71] Validation MSE: 0.78 
Epoch[72] Iteration[62800/879] Loss: 0.69
Epoch[72] Iteration[63200/879] Loss: 0.80
Epoch[72] Validation MSE: 0.78 
Epoch[73] Iteration[63600/879] Loss: 0.74
Epoch[73] Iteration[64000/879] Loss: 0.76
Epoch[73] Validation MSE: 0.78 
Epoch[74] Iteration[64400/879] Loss: 0.71
Epoch[74] Iteration[64800/879] Loss: 0.71
Epoch[74] Validation MSE: 0.78 
Epoch[75] Iteration[65200/879] Loss: 0.72
Epoch[75] Iteration[65600/879] Loss: 0.73
Epoch[75] Validation MSE: 0.77

Epoch[132] Validation MSE: 0.78 
Epoch[133] Iteration[116400/879] Loss: 0.77
Epoch[133] Iteration[116800/879] Loss: 0.71
Epoch[133] Validation MSE: 0.78 
Epoch[134] Iteration[117200/879] Loss: 0.68
Epoch[134] Iteration[117600/879] Loss: 0.77
Epoch[134] Validation MSE: 0.77 
Epoch[135] Iteration[118000/879] Loss: 0.76
Epoch[135] Iteration[118400/879] Loss: 0.72
Epoch[135] Validation MSE: 0.78 
Epoch[136] Iteration[118800/879] Loss: 0.70
Epoch[136] Iteration[119200/879] Loss: 0.73
Epoch[136] Validation MSE: 0.78 
Epoch[137] Iteration[119600/879] Loss: 0.61
Epoch[137] Iteration[120000/879] Loss: 0.71
Epoch[137] Iteration[120400/879] Loss: 0.76
Epoch[137] Validation MSE: 0.78 
Epoch[138] Iteration[120800/879] Loss: 0.73
Epoch[138] Iteration[121200/879] Loss: 0.78
Epoch[138] Validation MSE: 0.78 
Epoch[139] Iteration[121600/879] Loss: 0.78
Epoch[139] Iteration[122000/879] Loss: 0.76
Epoch[139] Validation MSE: 0.77 
Epoch[140] Iteration[122400/879] Loss: 0.73
Epoch[140] Iteration[122800/879]

Epoch[196] Iteration[172000/879] Loss: 0.71
Epoch[196] Validation MSE: 0.78 
Epoch[197] Iteration[172400/879] Loss: 0.66
Epoch[197] Iteration[172800/879] Loss: 0.77
Epoch[197] Validation MSE: 0.78 
Epoch[198] Iteration[173200/879] Loss: 0.68
Epoch[198] Iteration[173600/879] Loss: 0.78
Epoch[198] Iteration[174000/879] Loss: 0.79
Epoch[198] Validation MSE: 0.78 
Epoch[199] Iteration[174400/879] Loss: 0.78
Epoch[199] Iteration[174800/879] Loss: 0.68
Epoch[199] Validation MSE: 0.78 
Epoch[200] Iteration[175200/879] Loss: 0.67
Epoch[200] Iteration[175600/879] Loss: 0.83
Epoch[200] Validation MSE: 0.77 
Epoch[201] Iteration[176000/879] Loss: 0.73
Epoch[201] Iteration[176400/879] Loss: 0.68
Epoch[201] Validation MSE: 0.78 
Epoch[202] Iteration[176800/879] Loss: 0.69
Epoch[202] Iteration[177200/879] Loss: 0.70
Epoch[202] Validation MSE: 0.78 
Epoch[203] Iteration[177600/879] Loss: 0.67
Epoch[203] Iteration[178000/879] Loss: 0.74
Epoch[203] Iteration[178400/879] Loss: 0.75
Epoch[203] Validation

In [53]:
model.user.weight.data.numpy()

array([[ 3.2730633e-39, -2.4132871e-38,  3.2205121e-38, ...,
        -1.2124987e-39, -5.6631978e-39, -1.4563361e-38],
       [ 5.4120976e-01,  1.7743558e+00, -1.1254669e-01, ...,
         5.1624882e-01,  8.1065160e-01,  4.0898573e-01],
       [ 2.5255182e-01,  1.6480587e+00, -6.7774959e-02, ...,
         5.4440290e-01,  2.4153827e-01,  3.3421135e-01],
       ...,
       [-5.8217901e-01,  1.7236941e+00, -1.1019375e+00, ...,
        -4.0221814e-02, -5.8993435e-01, -5.1717963e-02],
       [-1.1850562e-02,  1.7629668e+00, -1.4603299e-01, ...,
         7.0980467e-02,  1.4373036e-01,  4.6519318e-01],
       [ 1.3147959e-01,  1.5219427e+00,  4.0817749e-02, ...,
        -1.0264318e+00, -7.9553372e-01,  2.1502918e-01]], dtype=float32)